In [2]:
# import librosa
import sys
sys.path.append('/Users/angelinazhai/ffmpeg') #https://stackoverflow.com/questions/57350259/filenotfounderror-errno-2-no-such-file-or-directory-ffprobe-ffprobe

# import scipy
from scipy import signal
import matplotlib.pyplot as plt
import os
import numpy as np

import librosa
import librosa.display

import pickle
import pandas as pd

import scipy
import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats

In [3]:
#relocate to folder with recordings
ROOT = '/Users/angelinazhai/Desktop/Github-Repositories/language-dsp'
ORIGINAL_REC = 'recordings/original'

In [4]:
#processing every single file in the folder

processed_info = {}

def update_dict():
    for root, dirs, files in os.walk(os.path.join(ROOT, ORIGINAL_REC)):
        for file in files:
            if file.endswith('.m4a'):
                y, sr = librosa.load(os.path.join(ROOT, ORIGINAL_REC, file))
                processed_info[file] = {}
                processed_info[file]['SR'] = sr
                processed_info[file]['Time_Domain'] = y
                D = librosa.stft(y)
                processed_info[file]['Freq_Domain'] = D
                S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                processed_info[file]['db'] = S_db


#pickle the processed info into new pickle file
def to_pickle_file():
    with open('processed_info.pickle', 'wb') as handle:
        pickle.dump(processed_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

# print(processed_info)


In [6]:
# plt.figure()
# librosa.display.specshow(S_db)
# plt.colorbar()
# plt.show()
# processed_info['Ree Eng.m4a']['Freq_Domain']

In [7]:
# #uncomment for double checking spectrogram

# plt.figure()
# librosa.display.specshow(processed_info['Anne Canto.m4a']['db'])
# plt.colorbar()
# plt.show()

In [8]:
#test opening pickle file
def dict_from_pickle():
    with open('processed_info.pickle', 'rb') as handle:
        b = pickle.load(handle)
    return b
    

In [9]:
#output test results
def output_results(diff, pval):
    print("Results: Percent difference in mean = {}, P-Value = {} ".format(abs(diff), pval))

In [10]:
# conduct t test on frequency information of two recordings
def t_test_freq(info, r1, r2):
    r1 = info[r1]['Freq_Domain']
    r2 = info[r2]['Freq_Domain']

    #flatten arrays
    r1 = r1.flatten()
    r2 = r2.flatten()

    t, p = scipy.stats.ttest_ind(r1, r2)
    return t, p


# conduct t test on time domain information of two recordings
def t_test_time(info, r1, r2):
    r1 = info[r1]['Time_Domain']
    r2 = info[r2]['Time_Domain']

    #flatten arrays
    r1 = r1.flatten()
    r2 = r2.flatten()

    t, p = scipy.stats.ttest_ind(r1, r2)
    return t, p



In [20]:
# find the variance of a dataset

#time domain
def variance_time(info, r):
    r = info[r]['Time_Domain']
    r = r.flatten()
    return np.var(r)

#frequency domain
def variance_freq(info, r):
    r = info[r]['Freq_Domain']
    r = r.flatten()
    return np.var(r)

**Loading daset from pickle**

In [ ]:
#read in dict from pickle file
info = dict_from_pickle()

**Comparing datasets in different domains by participants**

In [17]:
#output results for freq domain
print("Frequency domain analysis results:")

#conduct t test on two recordings (freq domain)
t1, p1 = t_test_freq(info, 'Ree Eng.m4a', 'Ree French.m4a')
output_results(t1, p1)

t2, p2 = t_test_freq(info, 'Ang English.m4a', 'Ang Shanghai.m4a')
output_results(t2, p2)

t3, p3 = t_test_freq(info, 'Nat English.m4a', 'Nat Indonesian .m4a') #there is a space
output_results(t3, p3)

t4, p4 = t_test_freq(info, 'Anne Eng.m4a', 'Anne Canto.m4a')
output_results(t4, p4)

Frequency domain analysis results:
Results: Percent difference in mean = 0.039417788713595824, P-Value = 0.9685573054218279 
Results: Percent difference in mean = 0.03469226472525642, P-Value = 0.972325135668208 
Results: Percent difference in mean = 0.0456987345100065, P-Value = 0.963550382531384 
Results: Percent difference in mean = 0.03816357871405698, P-Value = 0.9695572678180868 


In [1]:
#outpu results for time domain
print("Time domain analysis results:")

#conduct t test on two recordings (time domain)
t1, p1 = t_test_time(info, 'Ree Eng.m4a', 'Ree French.m4a')
output_results(t1, p1)

t2, p2 = t_test_time(info, 'Ang English.m4a', 'Ang Shanghai.m4a')
output_results(t2, p2)

t3, p3 = t_test_time(info, 'Nat English.m4a', 'Nat Indonesian .m4a') #there is a space
output_results(t3, p3)

t4, p4 = t_test_time(info, 'Anne Eng.m4a', 'Anne Canto.m4a')
output_results(t4, p4)

t5, p5 = t_test_time(info, 'Harshit English.mp3', 'Harshit Hindi.mp3')
output_results(t5, p5)

t6, p6 = t_test_time(info, 'Maisey English.m4a', 'Maisey Tagalog.m4a')
output_results(t6, p6)


Time domain analysis results:


NameError: name 't_test_time' is not defined

In [13]:
#conduct z test on two recordings (freq domain)
def z_test_freq(info, r1, r2):
    r1 = info[r1]['Freq_Domain']
    r2 = info[r2]['Freq_Domain']

    #flatten arrays
    r1 = r1.flatten()
    r2 = r2.flatten()

    z, p = statsmodels.stats.weightstats.ztest(r1, r2)
    return z, p

#conduct z test on two recordings (time domain)
def z_test_time(info, r1, r2):
    r1 = info[r1]['Time_Domain']
    r2 = info[r2]['Time_Domain']

    #flatten arrays
    r1 = r1.flatten()
    r2 = r2.flatten()

    z, p = statsmodels.stats.weightstats.ztest(r1, r2)
    return z, p

In [14]:
t1, p1 = z_test_freq(info, 'Ree Eng.m4a', 'Ree French.m4a')
print(abs(t1))

t2, p2 = z_test_freq(info, 'Ang English.m4a', 'Ang Shanghai.m4a')
print(abs(t2))

t3, p3 = z_test_freq(info, 'Nat English.m4a', 'Nat Indonesian .m4a') #there is a space
print(abs(t3))

t4, p4 = z_test_freq(info, 'Anne Eng.m4a', 'Anne Canto.m4a')
print(abs(t4))

0.013517177879344337
0.006396615196536191
0.012882250758857997
0.012071815958644448


In [15]:
t1, p1 = z_test_time(info, 'Ree Eng.m4a', 'Ree French.m4a')
print(abs(t1))

t2, p2 = z_test_time(info, 'Ang English.m4a', 'Ang Shanghai.m4a')
print(abs(t2))

t3, p3 = z_test_time(info, 'Nat English.m4a', 'Nat Indonesian .m4a') #there is a space
print(abs(t3))

t4, p4 = z_test_time(info, 'Anne Eng.m4a', 'Anne Canto.m4a')
print(abs(t4))

0.5444365239235549
0.2511470265391786
0.5074703814833658
0.48145604119837987


**Looking at Variance of Datasets**

(ie. Dispersion of datapoints within the dataset)

In [21]:
#calculating the variance of all recordings in time domain
print("Time domain variance results:")
print("Ree Eng.m4a: ", variance_time(info, 'Ree Eng.m4a'))
print("Ree French.m4a: ", variance_time(info, 'Ree French.m4a'))
print("Ang English.m4a: ", variance_time(info, 'Ang English.m4a'))
print("Ang Shanghai.m4a: ", variance_time(info, 'Ang Shanghai.m4a'))
print("Nat English.m4a: ", variance_time(info, 'Nat English.m4a'))
print("Nat Indonesian .m4a: ", variance_time(info, 'Nat Indonesian .m4a'))
print("Anne Eng.m4a: ", variance_time(info, 'Anne Eng.m4a'))
print("Anne Canto.m4a: ", variance_time(info, 'Anne Canto.m4a'))


Time domain variance results:
Ree Eng.m4a:  0.00035875867
Ree French.m4a:  0.00020856495
Ang English.m4a:  0.0013085506
Ang Shanghai.m4a:  0.0017567878
Nat English.m4a:  0.00032514118
Nat Indonesian .m4a:  0.0005495376
Anne Eng.m4a:  0.0006406574
Anne Canto.m4a:  0.00050900725


In [22]:
#calculating the variance of all recordings in freq domain
print("Frequency domain variance results:")
print("Ree Eng.m4a: ", variance_freq(info, 'Ree Eng.m4a'))
print("Ree French.m4a: ", variance_freq(info, 'Ree French.m4a'))
print("Ang English.m4a: ", variance_freq(info, 'Ang English.m4a'))
print("Ang Shanghai.m4a: ", variance_freq(info, 'Ang Shanghai.m4a'))
print("Nat English.m4a: ", variance_freq(info, 'Nat English.m4a'))
print("Nat Indonesian .m4a: ", variance_freq(info, 'Nat Indonesian .m4a'))
print("Anne Eng.m4a: ", variance_freq(info, 'Anne Eng.m4a'))
print("Anne Canto.m4a: ", variance_freq(info, 'Anne Canto.m4a'))


Frequency domain variance results:
Ree Eng.m4a:  0.29812798
Ree French.m4a:  0.16092809
Ang English.m4a:  1.0032241
Ang Shanghai.m4a:  1.3469301
Nat English.m4a:  0.25089777
Nat Indonesian .m4a:  0.4299686
Anne Eng.m4a:  0.533272
Anne Canto.m4a:  0.39134774
